In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from collections import Counter
from tqdm import tqdm
import numpy as np
import math
import os
import re
import pandas as pd
from typing import List, Tuple

In [2]:
def clean_text(text: str) -> str:
    text = str(text).lower()
    #i am removing contractions
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)

    text = re.sub(r"[^a-zA-Z\s]", " ", text) #i am removing special characters
     #i am removing extra spaces
    return text

In [3]:
def tokenize(text: str) -> List[str]:
    return clean_text(text).split()

In [4]:
class IMDbDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], vocab: dict, max_length: int):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx] 

        #i am tokenizing them and converting them to ids
        tokens = tokenize(text)
        token_ids = [self.vocab['<sos>']]
        token_ids.extend([self.vocab.get(token, self.vocab['<unk>']) for token in tokens])
        token_ids.append(self.vocab['<eos>'])

        #truncate or pad sequence
        if len(token_ids) > self.max_length:
            token_ids = token_ids[:self.max_length]
        else:
            padding = [self.vocab['<pad>']] * (self.max_length - len(token_ids))
            token_ids.extend(padding)

        return torch.tensor(token_ids), torch.tensor(label)

In [5]:
def build_vocab(texts: List[str], min_freq: int=5)->dict:
    counter = Counter()

    for text in tqdm(texts, desc="Building vocabulary"):
        tokens = tokenize(text)
        counter.update(tokens)

    vocab = {'<pad>':0,'<unk>':1,'<sos>':2,'<eos>':3} #special tokens
    idx = len(vocab)
    for token, count in counter.items():
        if count>=min_freq:
            vocab[token] = idx
            idx+=1
    return vocab

In [6]:
#the main task 
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, heads, dropout, mask=None):
        super().__init__()
        if(d_model %heads!=0): raise ValueError("d_model must be divisible by heads")
        self.d_model = d_model
        self.heads = heads
        self.mask = mask
        self.d_k = d_model // heads
        
        self.W_q = nn.Linear(d_model, d_model) #linear transformations are being performed here
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def split_heads(self, x):
        batch_size, seq_len, d_model = x.size()
        x = x.reshape(batch_size, seq_len, self.heads, self.d_k).transpose(1, 2)
        return x

    def combine_heads(self, x):
        batch_size, heads, seq_len, d_k = x.size()
        x = x.transpose(1, 2).reshape(batch_size, seq_len, self.d_model)
        return x
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf')) #will contribute nothing towards the token
        attention_weights = self.dropout(F.softmax(scores, dim=-1)) 
        attention_output = torch.matmul(attention_weights, V)
        return attention_output, attention_weights

    def forward(self,x,mask=None):
        Q = self.split_heads(self.W_q(x))
        K = self.split_heads(self.W_k(x))
        V = self.split_heads(self.W_v(x))
        attention_output,attention_weights = self.scaled_dot_product_attention(Q,K,V,mask)
        output = self.combine_heads(attention_output)
        output = self.W_o(output)
        return self.dropout(output)

In [7]:
class PositionWiseFeedForward(nn.Module): 
    def __init__(self, d_model, d_ff, dropout=0.1): #feed forward network
        super().__init__()
        self.linear1 = nn.Linear(d_model,d_ff) 
        self.linear2 = nn.Linear(d_ff,d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.dropout(x)
        return x

In [8]:
class LayerNorm(nn.Module):
    def __init__(self,d_model,eps=1e-12):
        super().__init__()
        self.gamma=nn.Parameter(torch.ones(d_model))
        self.beta=nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1,keepdim=True)
        var = x.var(-1,unbiased=False,keepdim=True)
        ans = self.gamma*(x-mean)/(var+self.eps).sqrt()+self.beta
        return ans

In [9]:
class EncoderLayer(nn.Module):
    def __init__(self,d_model,heads,d_ff,dropout):
        super().__init__()
        self.self_attention = MultiHeadSelfAttention(d_model,heads,dropout)
        self.feed_forward = PositionWiseFeedForward(d_model,d_ff,dropout)
        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x,mask=None):
        x_output = self.self_attention(x,mask)
        x = self.norm1(x+self.dropout(x_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,max_seq_length=5000):
        super().__init__()
        pe = torch.zeros(max_seq_length,d_model)
        position = torch.arange(0,max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()*(-math.log(10000.0)/d_model))
        
        pe[:,0::2] = torch.sin(position*div_term)
        pe[:,1::2] = torch.cos(position*div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer('pe',pe)
    def forward(self, x):
        return x + self.pe[:,:x.size(1)]

In [11]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model=512, n_heads=8, n_layers=6, d_ff=2048, dropout=0.1, max_seq_length=5000):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        self.norm = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self._init_parameters()
        
    def _init_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    
    def forward(self, x, mask=None):
        x = self.embedding(x) * math.sqrt(self.embedding.embedding_dim)
        x = self.positional_encoding(x)
        x = self.dropout(x)
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x, mask)
            
        return self.norm(x)

In [12]:
class TextTransformer(nn.Module): #feature extraction layer
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, num_classes, dropout=0.1, max_seq_length=512):
        super().__init__()
        self.transformer = Transformer(vocab_size, d_model, n_heads, n_layers, d_ff, dropout, max_seq_length)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, num_classes)
        )
    
    def forward(self, x, mask=None):
        x = self.transformer(x, mask)
        x = x.mean(dim=1)  #global average pooling is done in this case across seq_len
        return self.classifier(x)

In [13]:
def load_imdb_from_csv(csv_path: str) -> Tuple[List[str], List[int]]: #loaded imdb dataset from the csv
    print(f"Loading data from {csv_path}")
    df = pd.read_csv(csv_path)
    texts = df['review'].tolist()
    labels = (df['sentiment'] == 'positive').astype(int).tolist()
    
    return texts, labels

In [14]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    pbar = tqdm(train_loader, desc='Training')
    for batch_idx, (sequences, labels) in enumerate(pbar):
        sequences, labels = sequences.to(device), labels.to(device)
        mask = (sequences != 0).unsqueeze(1).unsqueeze(2)
        optimizer.zero_grad()
        outputs = model(sequences, mask)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': total_loss/(batch_idx+1),
            'acc': 100.*correct/total
        })
    
    return total_loss/len(train_loader), correct/total

In [15]:
def evaluate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for sequences, labels in tqdm(val_loader, desc='Evaluating'):
            sequences, labels = sequences.to(device), labels.to(device)
            mask = (sequences != 0).unsqueeze(1).unsqueeze(2)
            outputs = model(sequences, mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss/len(val_loader),correct/total

In [16]:
def predict_sentiment(model, text: str, vocab: dict, max_length: int, device: torch.device) -> str:
    model.eval()
    tokens = tokenize(text)
    token_ids = [vocab['<sos>']]
    token_ids.extend([vocab.get(token, vocab['<unk>']) for token in tokens])
    token_ids.append(vocab['<eos>'])
    if len(token_ids) > max_length:
        token_ids = token_ids[:max_length]
    else:
        padding = [vocab['<pad>']]*(max_length - len(token_ids))
        token_ids.extend(padding)
    with torch.no_grad():
        sequence = torch.tensor(token_ids).unsqueeze(0).to(device)
        mask = (sequence != 0).unsqueeze(1).unsqueeze(2)
        outputs = model(sequence, mask)
        prediction = torch.sigmoid(outputs[:, 1]).item()
        
        return "Positive" if prediction >= 0.5 else "Negative"

In [18]:
def main():
    #these are base configurations i am using
    MAX_LENGTH = 512
    BATCH_SIZE = 32
    D_MODEL = 512
    N_HEADS = 8
    N_LAYERS = 1
    D_FF = 2048
    DROPOUT = 0.1
    LEARNING_RATE = 1e-4
    NUM_EPOCHS = 5
    MIN_FREQ = 5
    NUM_CLASSES = 2
    TRAIN_RATIO = 0.8
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"using device: {device}")
    csv_path = "IMDB_Dataset.csv" #path of the dataset
    texts, labels = load_imdb_from_csv(csv_path)
    
    print("building vocabulary")
    vocab = build_vocab(texts, min_freq=MIN_FREQ)
    vocab_size = len(vocab)
    print(f"vocabulary size: {vocab_size}")
    
    dataset = IMDbDataset(texts, labels, vocab, MAX_LENGTH)
    
    train_size = int(TRAIN_RATIO * len(dataset))
    val_size = len(dataset)-train_size
    train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    model = TextTransformer(
        vocab_size=vocab_size,
        d_model=D_MODEL,
        n_heads=N_HEADS,
        n_layers=N_LAYERS,
        d_ff=D_FF,
        num_classes=NUM_CLASSES,
        dropout=DROPOUT,
        max_seq_length=MAX_LENGTH
    ).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE) #Adam optimiser is working better than sgd
    
    best_val_acc = 0.0
    for epoch in range(NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device) #training
        print(f"Training - Loss: {train_loss:.4f}, Accuracy: {train_acc*100:.2f}%")
        
        val_loss, val_acc = evaluate(model, val_loader, criterion, device) #evaluation
        print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.2f}%")
        
    test_text = "This movie was absolutely fantastic! The acting was superb and the story was engaging throughout."
    sentiment = predict_sentiment(model, test_text, vocab, MAX_LENGTH, device)
    print(f"\nexample prediction:")
    print(f"text: {test_text}")
    print(f"predicted sentiment: {sentiment}")

if __name__ == "__main__":
    main()

using device: cpu
Loading data from IMDB_Dataset.csv
building vocabulary


Building vocabulary: 100%|██████████| 50000/50000 [00:03<00:00, 14802.39it/s]


vocabulary size: 39414

Epoch 1/5


Training:  63%|██████▎   | 782/1250 [29:52<17:52,  2.29s/it, loss=0.38, acc=81.2] 


KeyboardInterrupt: 